In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 30.7 MB/s eta 0:00:00


In [2]:
from ultralytics import YOLO
print("YOLO installed successfully")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
YOLO installed successfully


In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"dhinanjayafernando","key":"2de418079f31d879c8d3bf5be3da4cae"}'}

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets list | head

ref                                                     title                                           size  lastUpdated                 downloadCount  voteCount  usabilityRating  
------------------------------------------------------  ----------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
neurocipher/heartdisease                                Heart Disease                                   3491  2025-12-11 15:29:14.327000           2114        168  1.0              
neurocipher/student-performance                         Student Performance                            49705  2025-12-12 12:06:28.973000           1261        105  1.0              
dansbecker/powerlifting-database                        powerlifting-database                        9277600  2019-04-30 21:07:41.560000          24255        351  0.5882353        
rtatman/188-million-us-wildfires                        1.88 Million US Wildfires         

In [ ]:
!kaggle datasets download -d kmader/malaria-bounding-boxes

Dataset URL: https://www.kaggle.com/datasets/kmader/malaria-bounding-boxes
License(s): Attribution-NonCommercial-ShareAlike 3.0 IGO (CC BY-NC-SA 3.0 IGO)
 16% 698M/4.21G [00:06<00:51, 73.1MB/s]

In [ ]:
!unzip malaria-bounding-boxes.zip -d malaria_data

In [ ]:
!ls malaria_data/malaria

In [ ]:
import cv2
import matplotlib.pyplot as plt
import os

image_dir = "malaria_data/malaria/images"
image_files = os.listdir(image_dir)

sample_img = cv2.imread(os.path.join(image_dir, image_files[0]))
sample_img = cv2.cvtColor(sample_img, cv2.COLOR_BGR2RGB)

plt.imshow(sample_img)
plt.title("Sample Blood Smear Image")
plt.axis("off")


In [ ]:
!pip install pycocotools

In [ ]:
import os

base_dir = "yolo_malaria"
os.makedirs(base_dir + "/images/train", exist_ok=True)
os.makedirs(base_dir + "/labels/train", exist_ok=True)

In [ ]:
import json

with open("malaria_data/malaria/training.json", "r") as f:
    data = json.load(f)

print(type(data))
print(data[0].keys())

In [ ]:
ann = data[0]
print(ann)

print("\nFirst object:")
print(ann["objects"][0])

In [ ]:
import json
import os
import shutil
from PIL import Image

# Paths
json_path = "malaria_data/malaria/training.json"
image_root = "malaria_data/malaria"
out_img_dir = "yolo_malaria/images/train"
out_lbl_dir = "yolo_malaria/labels/train"

os.makedirs(out_img_dir, exist_ok=True)
os.makedirs(out_lbl_dir, exist_ok=True)

# Load annotations
with open(json_path, "r") as f:
    annotations = json.load(f)

for ann in annotations:
    # Extract image info
    img_rel_path = ann["image"]["pathname"].lstrip("/")
    img_path = os.path.join(image_root, img_rel_path)
    img_name = os.path.basename(img_path)

    # Copy image
    shutil.copy(img_path, os.path.join(out_img_dir, img_name))

    image = Image.open(img_path)
    img_w, img_h = image.size

    label_lines = []

    for obj in ann["objects"]:
        # ONLY malaria parasites
        if obj["category"] != "trophozoite":
            continue

        bb = obj["bounding_box"]
        r_min = bb["minimum"]["r"]
        c_min = bb["minimum"]["c"]
        r_max = bb["maximum"]["r"]
        c_max = bb["maximum"]["c"]

        # Convert to x, y, w, h
        x = c_min
        y = r_min
        w = c_max - c_min
        h = r_max - r_min

        # YOLO normalization
        x_center = (x + w / 2) / img_w
        y_center = (y + h / 2) / img_h
        w /= img_w
        h /= img_h

        label_lines.append(f"0 {x_center} {y_center} {w} {h}")

    # Write label file ONLY if parasites exist
    if label_lines:
        label_file = img_name.rsplit(".", 1)[0] + ".txt"
        with open(os.path.join(out_lbl_dir, label_file), "w") as f:
            f.write("\n".join(label_lines))

In [ ]:
!ls yolo_malaria/labels/train | head

In [ ]:
!cat yolo_malaria/labels/train/$(ls yolo_malaria/labels/train | head -n 1)

In [ ]:
data_yaml = """
path: yolo_malaria
train: images/train
val: images/train

names:
  0: trophozoite
"""

with open("yolo_malaria/data.yaml", "w") as f:
    f.write(data_yaml)

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")

model.train(
    data="yolo_malaria/data.yaml",
    epochs=50,
    imgsz=640,
    batch=16,
    patience=10
)

In [ ]:
!ls runs/detect/train/weights

In [ ]:
from ultralytics import YOLO

model = YOLO("runs/detect/train/weights/best.pt")

In [ ]:
test_image = "malaria_data/malaria/images/005e60b6-77b8-458c-b57c-bfe0c7e7df78.png"

In [ ]:
results = model(test_image, conf=0.25)

In [ ]:
results[0].show()

In [ ]:
detections = results[0].boxes
parasite_count = len(detections)

print("Detected malaria parasites:", parasite_count)

In [ ]:
metrics = model.val()
print("Precision:", metrics.box.mp)
print("Recall:", metrics.box.mr)
print("mAP50:", metrics.box.map50)
print("mAP50-95:", metrics.box.map)